In [9]:
from pathlib import Path
import requests


from scraper import create_dataframe
from spotify_api import headers

In [10]:
df, names = create_dataframe(Path("../league"))

In [ ]:
tracks = ','.join(df["song_id"].iloc[:])
url = f'https://api.spotify.com/v1/tracks?ids={tracks}'

p = requests.get(url, headers=headers)

In [ ]:
len(p.json()["tracks"])

In [17]:
x.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [22]:
x["artists"], x["duration_ms"], x["explicit"], x["name"], x["popularity"]

([{'external_urls': {'spotify': 'https://open.spotify.com/artist/7Ead768rc4ShGxnqtqccU5'},
   'href': 'https://api.spotify.com/v1/artists/7Ead768rc4ShGxnqtqccU5',
   'id': '7Ead768rc4ShGxnqtqccU5',
   'name': 'Sarah Brightman',
   'type': 'artist',
   'uri': 'spotify:artist:7Ead768rc4ShGxnqtqccU5'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/5PbgBNZiQUNOh5JEVQtfH3'},
   'href': 'https://api.spotify.com/v1/artists/5PbgBNZiQUNOh5JEVQtfH3',
   'id': '5PbgBNZiQUNOh5JEVQtfH3',
   'name': 'Hot Gossip',
   'type': 'artist',
   'uri': 'spotify:artist:5PbgBNZiQUNOh5JEVQtfH3'}],
 258986,
 False,
 'I Lost My Heart to a Starship Trooper',
 26,
 'track')

In [25]:
x["artists"][1]["id"]

'5PbgBNZiQUNOh5JEVQtfH3'